# Download Twitter Data

#### Import Dependencies

In [ ]:
# Tweepy library to connect to the Twitter API and process the response
import tweepy
# JSON to serialise JSON
import json
# Pandas to work with dataframes
import pandas as pd
# wget to retrieve profile images
import wget
# OS to manipulate file paths and names
import os
# Shutil to handle file duplicates
import shutil
# Time to handle API rate limiting
import time

# Read environment variables
from dotenv import load_dotenv

#### Load configuration file for environment variables

In [ ]:
load_dotenv('configuration.env')

#### Load Twitter authentication from environment variables

In [ ]:
# Twitter App credentials
consumer_key = os.getenv('TWITTER_CONSUMER_KEY')
consumer_secret = os.getenv('TWITTER_CONSUMER_SECRET')
access_token = os.getenv('TWITTER_ACCESS_TOKEN')
access_token_secret = os.getenv('TWITTER_ACCESS_TOKEN_SECRET')

#### Twitter Authentication

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, retry_count=3,retry_delay=5,retry_errors=set([401, 404, 500, 503]))

#### Set API Rate Limit

In [ ]:
rate_limit = 5

#### Seed User

In [ ]:
# User's screen name
screen_name = 'BIM360'

### Download Twitter Friends

Function for downloading a user's friends (the accounts they follow). The function requires a user's screen name. It also accepts optional arguments to filter friends from results based on a number of criteria.

- `profile_image_only=True` excludes friends without a profile image (using the Twitter default). 
- `url_only=True` excludes friends without a URL specified on their profile.

**Note:** Even if the image or url filters are set, Tweepy still makes an API call to check these accounts. They are excluded from the recorded results but still contribute to the intitial number of API calls. The value of these filters is that they reduce the the pool of friends and corresponding number of API calls required when undertaking subsequent searches for friends-of-friends, or friends-of-friends-of-friends etc.

In [ ]:
def download_friends(screen_name, profile_image_only=False, url_only=False):    
    
    print('Processing friends of ' + screen_name)
    
    # Count of processed friends
    count = 0
    # List of friends
    friends = []

    # Iterate through friends
    for user in tweepy.Cursor(api.friends, screen_name).items():
        
        # Increment friend count
        count = count + 1
        print(str(count) + ': ' + user.screen_name)
        
        # Filter friends not matching specified criteria
        if (profile_image_only==True and '/default_profile_images/default_profile_normal.png' in user.profile_image_url):
            print('Skipping user with default profile image')
            # Wait for rate limiter
            time.sleep(rate_limit)
            continue
        elif (url_only==True and 'url' not in user.entities):
            print('Skipping user without associated url')
            # Wait for rate limiter
            time.sleep(rate_limit)
            continue
        
        #Store the user's data
        data = {'id_str': user.id_str,
                'screen_name': user.screen_name,
                'name': user.name,
                'location': user.location,
                'profile_image_url': user.profile_image_url,
                'description': user.description,
                # Placeholder for website url
                'expanded_url': '',
                'followers_count': user.followers_count,
                'friends_count': user.friends_count,
                # Format datetime object
                'created_at': user.created_at.strftime("%y-%m-%d, %H:%M:%S"),
                'verified': user.verified
               }
        
        # Check for website URL and add to data
        if ('url' in user.entities):
            data['expanded_url'] = user.entities['url']['urls'][0]['expanded_url'],
        else:
            print(user.screen_name + ' has no associated url')
            
        # Save the Twitter profile image
        file_destination = os.path.join('logos/' + user.screen_name + '.png')        
        # Check if file already exists
        if os.path.exists(file_destination) != True:
            try:
                # Remove the '_normal' tag from the URL to get a full sized image
                link = user.profile_image_url.replace('_normal','')
                # Use wget to download and save the image file
                wget.download(link, file_destination)
            except:
                print('Profile image not available')
        else:
            print('Profile image already downloaded')
        
        # Append user to list of friends
        friends.append(data)
        
        # Wait for rate limiter
        time.sleep(rate_limit)
        
    print(screen_name + ' processing complete!')
    return friends

#### Download friends of seed user

In [ ]:
friends = download_friends(screen_name)

with open('./friends_lists/{screen_name}.json'.format(screen_name=screen_name), 'w') as file:
    file.write(json.dumps(friends))

#### Download friends-of-friends

In [ ]:
for friend in friends:
    screen_name = friend['screen_name']
    
    fof = download_friends(screen_name)
    
    with open('./friends_lists/{screen_name}.json'.format(screen_name=screen_name), 'w') as file:
        file.write(json.dumps(fof))
        
print('Downloading friends-of-friends complete!')